SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (1544676997.py, line 1)

In [111]:
import os
import pathlib
import numpy as np
import tensorflow as tf
print(os.path.sep)

\


In [64]:
# Get the list of file path of a train image in the data_dir.
data_dir = "../data/mvtec"
data_dir = pathlib.Path(data_dir)
list_train = list(data_dir.glob("*/train/good/*.png"))
list_test = list(data_dir.glob("*/test/*/*.png"))
num_train = len(list_train)  # the number of train images.
num_test = len(list_test)

# Create train/test dataset. Each element in the dataset is path of image file.
list_train_ds = tf.data.Dataset.list_files(str(data_dir/"*/train/good/*.png"), shuffle=False)
list_test_ds = tf.data.Dataset.list_files(str(data_dir/"*/test/*/*.png"), shuffle=False)

# Shuffle these dataset once.
list_train_ds = list_train_ds.shuffle(buffer_size=num_train, reshuffle_each_iteration=False)
list_test_ds = list_test_ds.shuffle(buffer_size=num_test, reshuffle_each_iteration=False)

list_train_ds_s = list_train_ds.skip(int(0.9 * num_train))
list_train_ds_l = list_train_ds.take(int(0.9 * num_train))
list_test_ds_s = list_test_ds.skip(int(0.9 * num_test))
list_test_ds_l = list_test_ds.take(int(0.9 * num_test))

print(f"list_train_s count: {list_train_ds_s.cardinality()} \n list_train_l count: {list_train_ds_l.cardinality()}")
print(f"list_test_s count: {list_test_ds_s.cardinality()} \n list_test_l count: {list_test_ds_l.cardinality()}")

print(f"train count: {num_train} , test count: {num_test}")

list_train_s count: 363 
 list_train_l count: 3266
list_test_s count: 173 
 list_test_l count: 1552
train count: 3629 , test count: 1725


In [65]:
train_ds = list_train_ds_l
finetune_ds = list_train_ds_s.concatenate(list_test_ds_s)
eval_ds = list_test_ds_l

print(f"fine_tune_ds: {finetune_ds.cardinality().numpy()}")

fine_tune_ds: 536


In [61]:
class_names = [item for item in data_dir.glob("*") if item.is_dir()]
class_names

[WindowsPath('../data/mvtec/bottle'),
 WindowsPath('../data/mvtec/cable'),
 WindowsPath('../data/mvtec/capsule'),
 WindowsPath('../data/mvtec/carpet'),
 WindowsPath('../data/mvtec/grid'),
 WindowsPath('../data/mvtec/hazelnut'),
 WindowsPath('../data/mvtec/leather'),
 WindowsPath('../data/mvtec/metal_nut'),
 WindowsPath('../data/mvtec/pill'),
 WindowsPath('../data/mvtec/screw'),
 WindowsPath('../data/mvtec/tile'),
 WindowsPath('../data/mvtec/toothbrush'),
 WindowsPath('../data/mvtec/transistor'),
 WindowsPath('../data/mvtec/wood'),
 WindowsPath('../data/mvtec/zipper')]

In [72]:
parts = tf.strings.split(next(train_ds.take(1)).numpy(), os.path.sep)

TypeError: 'TakeDataset' object is not an iterator

In [89]:
file_path = list(train_ds.as_numpy_iterator())[:5]
file_path

[b'..\\data\\mvtec\\screw\\test\\good\\035.png',
 b'..\\data\\mvtec\\tile\\test\\crack\\000.png',
 b'..\\data\\mvtec\\hazelnut\\test\\cut\\013.png',
 b'..\\data\\mvtec\\metal_nut\\test\\bent\\002.png',
 b'..\\data\\mvtec\\screw\\test\\thread_side\\016.png']

In [114]:
file_path = list(train_ds.as_numpy_iterator())[:5]
parts = tf.strings.split(file_path, os.path.sep)
sub_dir = parts[:, -2:-1].numpy()
print(sub_dir.squeeze())
# for part in parts:
#     print(part.numpy())
label = (sub_dir == b'good').astype(np.int32)
label.shape

[b'good' b'crack' b'cut' b'bent' b'thread_side']


(5, 1)

In [121]:
file_path = list(train_ds.as_numpy_iterator())[1]
file_path

b'..\\data\\mvtec\\tile\\test\\crack\\000.png'

In [122]:
def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    label = int(parts[-2] == b'good')
    return label
get_label(file_path)

0